In [1]:
from PIL import Image 
import PIL 
from pdf2image import convert_from_path
from ocrmac import ocrmac
import os
import threading
import PyPDF2
import pandas as pd
from unidecode import unidecode
import re
import ast
from neo4j import GraphDatabase
from credentials import *

In [2]:
def search(seekable_name):
    extract_files = os.listdir("extracts/")
    try:
        extract_files.remove(".DS_Store")
    except:
        pass
    extract_files.sort()

    try:
        with open(f"result/{seekable_name}.html", "w") as f:
            f.write("")
    except:
        pass
    
    uni_seekable_name = unidecode(seekable_name).lower()

    print(uni_seekable_name)

    if "," in uni_seekable_name:
        nom = uni_seekable_name.split(",")[0].strip()
        prenoms = uni_seekable_name.split(",")[1].strip()
        prenoms = re.sub(r"[ \-\(\)\/\.]", " ", prenoms)
        prenoms = re.sub(" +", " ", prenoms)
        prenoms = prenoms.strip().split(" ")
        prenoms = [prenom for prenom in prenoms if len(prenom) > 2] # suppression si juste initiales
    # else:
    #     nom = uni_seekable_name.strip()
    #     prenom = []

    with open(f"results2/{seekable_name}.html", "a") as f:
        for extract_file in extract_files:
            year = extract_file[0:4]
            title_year = f"<h1>{year}</h1>"
            f.write(f"\n{title_year}\n\n")
            df = pd.read_csv(f"extracts/{extract_file}")

            for i in range(len(df)):
                lignes = ast.literal_eval(df["text"][i])
                num_page = df["page"][i]
                page_link = f"<a href='file:///Users/antoinelavacquery/Annuaires/{year}.pdf#page={num_page}'>page {num_page}</a>"

                for ligne in lignes:
                    lower = ligne.lower()
                    uni = unidecode(lower)

                    if "," in uni_seekable_name and prenoms:
                        found = False
                        for prenom in prenoms:
                            if prenom in uni and nom in uni:
                                found = True
                        if found:
                            f.write(f"{ligne} -> {page_link}<br>\n")
                    # else:
                    #     if nom in uni:
                    #         f.write(f"{ligne} -> {page_link}<br>\n")

In [3]:
def get_neo4j_results_of(query):
    URI = AURA_CONNECTION_URI
    USER = AURA_USERNAME
    PASS = AURA_PASSWORD

    # Driver instantiation
    driver = GraphDatabase.driver(URI, auth=(USER, PASS))

    # Create a driver session
    with driver.session() as session:
        # Use .data() to access the results array
        results = session.run(query)
        results_data = results.data()
        return(results_data)
    driver.close()

In [4]:
query = """
MATCH (h:HOMME)-[e:EXERCE]->(a:ACTIVITÉ {nom:'Agent de change'})
WHERE (e.date_début.year >= 1815 AND e.date_début.year <= 1852)
OR (e.date_fin.year >= 1815 AND e.date_fin.year <= 1852)
RETURN h
"""

results = get_neo4j_results_of(query)
all = [result["h"]["nom"] for result in results]

for name in all:
    search(name)

# search("Bouchet, Benoît")

bouchet, benoit
